In [1]:
from os.path import expanduser
import pandas as pd
import simplejson
import numpy as np
import urllib
import json

# Load metadata

In [6]:
new_meta_frame = pd.read_csv('../data/meta_2015_with_elev.csv')

In [7]:
new_meta_frame.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Unnamed: 0.1.1', u'ID', u'Fwy',
       u'Dir', u'District', u'County', u'City', u'State_PM', u'Abs_PM',
       u'Latitude', u'Longitude', u'Length', u'Type', u'Lanes', u'Name',
       u'User_ID_1', u'User_ID_2', u'User_ID_3', u'User_ID_4', u'file_date',
       u'elevation'],
      dtype='object')

In [8]:
new_meta_frame.Type.unique()

array(['ML', 'OR', 'FR', 'FF', 'HV', 'CH', 'CD'], dtype=object)

In [20]:
def create_freeway_vectors(frame_to_use, columns_to_select=['ID', 'Latitude', 'Longitude', 'Abs_PM', 'Lanes']):
    to_loop = new_meta_frame.groupby(['Fwy', 'Dir'])['ID'].count().reset_index()[['Fwy', 'Dir']].values
    
    ret = {}
    for Fwy, Dir in to_loop:
        if Dir == "N":
            sort_order = ('Abs_PM', True)        
        elif Dir == "S":
            sort_order = ('Abs_PM', True)        
        elif Dir == "E":
            sort_order = ('Abs_PM', True)        
        elif Dir == "W":
            sort_order = ('Abs_PM', True)
            
        tmp = frame_to_use[(frame_to_use.Fwy == Fwy) & (frame_to_use.Dir == Dir)]\
            .sort_values(by=sort_order[0], ascending=sort_order[1])[columns_to_select].drop_duplicates()
        tmp['order'] = pd.Series(index=tmp.index, data=sorted(range(0, len(tmp.ID)), reverse=(not sort_order[1])))
        ret["%s_%s" % (Fwy, Dir)] = tmp
    return ret 

In [21]:
def sorted_func(x):
    values = x.split('_')
    Fwy = int(values[0])
    Dir = values[1]
    if Dir == 'N' or Dir == 'E':
        dir_weight = 0
    else:
        dir_weight = 1
    return Fwy + dir_weight

In [25]:
freeway_vectors_update = create_freeway_vectors(
    new_meta_frame, [u'ID', u'Fwy', u'Dir', u'Abs_PM', u'Latitude', u'Longitude', u'Lanes', u'Name', 'Type',
     u'elevation'])

In [26]:
freeway_keys = sorted(freeway_vectors_update.keys(), key=sorted_func)
freeway_keys

['5_N',
 '5_S',
 '8_E',
 '8_W',
 '15_N',
 '15_S',
 '52_E',
 '52_W',
 '54_E',
 '54_W',
 '56_E',
 '56_W',
 '67_N',
 '67_S',
 '78_E',
 '78_W',
 '94_E',
 '94_W',
 '125_N',
 '125_S',
 '163_N',
 '163_S',
 '805_N',
 '805_S',
 '905_E',
 '905_W']

In [27]:
freeway_vectors_update['5_N']

,ID,Fwy,Dir,Abs_PM,Latitude,Longitude,Lanes,Name,Type,elevation,order
847,1114091,5,N,0.057,32.542842,-117.030331,6,N/O CMNO DE LA PLAZA,ML,19.185152,0
1064,1118333,5,N,1.143,32.551690,-117.045725,4,5 NB @ S. Ysidro,ML,17.266630,1
890,1114709,5,N,1.412,32.553384,-117.049868,4,N/O VIA DE SAN YSIDR,ML,15.447632,2
1066,1118348,5,N,2.195,32.558459,-117.061845,4,5 NB @ Dairy Mart,ML,12.569578,3
891,1114720,5,N,2.560,32.561334,-117.067081,4,N/O DAIRY MART RD,ML,11.265139,4
1067,1118352,5,N,3.335,32.569052,-117.076507,4,5 NB N/O 905,ML,10.706306,5
1101,1118743,5,N,4.004,32.577106,-117.082889,4,5 NB @ Coronado Ave,ML,9.587918,6
1070,1118379,5,N,4.578,32.584267,-117.087224,4,5 NB @ Palm Ave,ML,4.707740,7
893,1114734,5,N,5.230,32.593827,-117.088439,5,S/O MAIN ST,ML,6.593952,8
1396,1123135,5,N,5.324,32.595086,-117.088638,1,5 NB off to Main St,ML,7.442711,9


In [32]:
for ind, i in freeway_vectors_update['5_N'].iterrows():
    print i
    print i['ID']
    break

ID                        1114091
Fwy                             5
Dir                             N
Abs_PM                      0.057
Latitude                  32.5428
Longitude                 -117.03
Lanes                           6
Name         N/O CMNO DE LA PLAZA
Type                           ML
elevation                 19.1852
order                           0
Name: 847, dtype: object
1114091


In [34]:
final = {}

for key in freeway_keys:
    new_geojson = {'type': 'FeatureCollection', 'features': []}

    for idx, row in freeway_vectors_update[key].iterrows():
        properties = {'key': key,
                      'ID': row['ID'],
                      'Lanes': row['Lanes'],
                      'Name': row['Name'],
                      'Abs_PM': row['Abs_PM'],
                      'Elevation': row['elevation'],
                      'Order': row['order'],
                      'Type': row['Type']
                     }
        geometry = {'type': "Point", "coordinates": [row['Longitude'], row['Latitude']]}
        temp = {'type': 'Feature', 'properties': properties, "geometry": geometry}
        new_geojson['features'].append(temp)

    print "geojson len: %s" % len(new_geojson['features'])
    final[key] = {'visible': False, 'data': new_geojson}
json.dump(final, open('../data/2015_d11_geojson_points.json', 'w'))

geojson len: 171
geojson len: 182
geojson len: 66
geojson len: 88
geojson len: 153
geojson len: 149
geojson len: 23
geojson len: 23
geojson len: 8
geojson len: 8
geojson len: 27
geojson len: 24
geojson len: 2
geojson len: 2
geojson len: 46
geojson len: 62
geojson len: 36
geojson len: 53
geojson len: 48
geojson len: 51
geojson len: 24
geojson len: 29
geojson len: 117
geojson len: 112
geojson len: 17
geojson len: 20
